# Functions and Imports - no pyiron dependence

In [1]:
from h5io_browser.base import read_nested_dict_from_hdf, list_hdf, write_dict_to_hdf
import os

In [2]:
def get_node_from_job_dict(job_dict, node):
    node_name_lst = node.split("/")
    tmp_dict = job_dict
    for group in node_name_lst:
        tmp_dict = tmp_dict[group]
    return tmp_dict

In [3]:
def convert_generic_parameters_to_string(generic_parameter_dict):
    output_str = ""
    for p, v in zip(generic_parameter_dict["data_dict"]["Parameter"], generic_parameter_dict["data_dict"]["Value"]):
        output_str += p.replace("___", " ") + " " + str(v) + "\n"
    return output_str[:-1]

In [4]:
def convert_generic_parameters_to_dictionary(generic_parameter_dict):
    return {p: v for p, v in zip(generic_parameter_dict["data_dict"]["Parameter"], generic_parameter_dict["data_dict"]["Value"])}

In [5]:
def filter_dict(input_dict, remove_keys_lst):
    def recursive_filter(input_value, remove_keys_lst):
        if isinstance(input_value, dict):
            return filter_dict(input_dict=input_value, remove_keys_lst=remove_keys_lst)
        else:
            return input_value 
    
    return {k: recursive_filter(input_value=v, remove_keys_lst=remove_keys_lst) for k, v in input_dict.items() if k not in remove_keys_lst}

In [6]:
def sort_dictionary_from_datacontainer(input_dict):
    def recursive_sort(input_value):
        if isinstance(input_value, dict):
            return sort_dictionary_from_datacontainer(input_dict=input_value)
        else:
            return input_value 
    
    ind_dict, content_dict = {}, {}
    content_lst_flag = False
    for k, v in input_dict.items():
        if "__index_" in k:
            key, ind = k.split("__index_")
            if key ==  "":
                content_lst_flag = True
                ind_dict[int(ind)] = recursive_sort(input_value=v)               
            else:
                ind_dict[int(ind)] = key
                content_dict[key] = recursive_sort(input_value=v)
        else:
            content_dict[k] = recursive_sort(input_value=v)
    if content_lst_flag:
        return [ind_dict[ind] for ind in sorted(list(ind_dict.keys()))]
    elif len(ind_dict) == len(content_dict):
        return {ind_dict[ind]: content_dict[ind_dict[ind]] for ind in sorted(list(ind_dict.keys()))}
    elif len(ind_dict) == 0: 
        return content_dict
    else:
        raise KeyError()

In [7]:
def convert_datacontainer_to_dictionary(data_container_dict):
    return sort_dictionary_from_datacontainer(input_dict=filter_dict(
        input_dict=data_container_dict, 
        remove_keys_lst=["NAME", "TYPE", "OBJECT", "HDF_VERSION", "READ_ONLY", "VERSION"],
    ))

# Create VASP job

In [8]:
from pyiron_atomistics import Project

In [9]:
pr = Project("test")

In [10]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [12]:
len(structure)

4

In [13]:
job = pr.create.job.Vasp("vasp")

In [14]:
job.structure =structure

In [15]:
job.calc_minimize()

In [16]:
job.run()

The job vasp was saved and received the ID: 22808001


# Read job dictionary

In [17]:
job_dict = read_nested_dict_from_hdf(
    file_name=job.project_hdf5.file_name,
    h5_path="/",
    recursive=True,
    slash='ignore',
)

In [18]:
convert_generic_parameters_to_string(generic_parameter_dict=job_dict["vasp"]["input"]["incar"]).split("\n")

['SYSTEM vasp',
 'PREC Accurate',
 'ALGO Fast',
 'LREAL False',
 'LWAVE False',
 'LORBIT 0',
 'ISIF 2',
 'IBRION 2',
 'NELM 400',
 'NSW 100',
 'EDIFFG -0.01']

In [19]:
convert_generic_parameters_to_string(generic_parameter_dict=job_dict["vasp"]["input"]["kpoints"]).split("\n")

[' Kpoints file generated with pyiron_atomistics',
 ' 0',
 ' Monkhorst_Pack',
 ' 4 4 4',
 ' 0 0 0']

In [20]:
convert_datacontainer_to_dictionary(data_container_dict=job_dict["vasp"]["executable"]["executable"])

{'version': '5.4.4',
 'name': 'vasp',
 'operation_system_nt': False,
 'executable': None,
 'mpi': False,
 'accepted_return_codes': [0]}

In [21]:
job_dict["vasp"]["output"]

{'description': 'This contains all the output static from this particular vasp run',
 'charge_density': {'TYPE': "<class 'pyiron_atomistics.vasp.volumetric_data.VaspVolumetricData'>",
  'total': array([[[-0.01030276, -0.00409825,  0.01385827, ...,  0.04154952,
            0.01385827, -0.00409825],
          [-0.00409825,  0.00203889,  0.0197738 , ...,  0.04704767,
            0.0197738 ,  0.00203889],
          [ 0.01385827,  0.0197738 ,  0.03679736, ...,  0.06277546,
            0.03679736,  0.0197738 ],
          ...,
          [ 0.04154952,  0.04704767,  0.06277546, ...,  0.08650493,
            0.06277546,  0.04704767],
          [ 0.01385827,  0.0197738 ,  0.03679736, ...,  0.06277546,
            0.03679736,  0.0197738 ],
          [-0.00409825,  0.00203889,  0.0197738 , ...,  0.04704767,
            0.0197738 ,  0.00203889]],
  
         [[-0.00409825,  0.00203889,  0.0197738 , ...,  0.04704767,
            0.0197738 ,  0.00203889],
          [ 0.00203889,  0.0081043 ,  0.025606

# Get VASP schema

In [22]:
file_name = job.project_hdf5.file_name
file_name

'/cmmc/u/janj/notebooks/2024/2024-06-06-read-hdf/test/vasp.h5'

In [23]:
node_lst = list_hdf(file_name=file_name, h5_path="/", recursive=True)[0]
node_lst

['/vasp/DICT_VERSION',
 '/vasp/HDF_VERSION',
 '/vasp/NAME',
 '/vasp/OBJECT',
 '/vasp/TYPE',
 '/vasp/VERSION',
 '/vasp/executable',
 '/vasp/job_id',
 '/vasp/server',
 '/vasp/status',
 '/vasp/input/generic_dict',
 '/vasp/input/interactive',
 '/vasp/input/potential_dict',
 '/vasp/input/generic/DICT_VERSION',
 '/vasp/input/generic/NAME',
 '/vasp/input/generic/OBJECT',
 '/vasp/input/generic/TYPE',
 '/vasp/input/generic/VERSION',
 '/vasp/input/generic/data_dict',
 '/vasp/input/incar/DICT_VERSION',
 '/vasp/input/incar/NAME',
 '/vasp/input/incar/OBJECT',
 '/vasp/input/incar/TYPE',
 '/vasp/input/incar/VERSION',
 '/vasp/input/incar/data_dict',
 '/vasp/input/kpoints/DICT_VERSION',
 '/vasp/input/kpoints/NAME',
 '/vasp/input/kpoints/OBJECT',
 '/vasp/input/kpoints/TYPE',
 '/vasp/input/kpoints/VERSION',
 '/vasp/input/kpoints/data_dict',
 '/vasp/input/potcar/DICT_VERSION',
 '/vasp/input/potcar/NAME',
 '/vasp/input/potcar/OBJECT',
 '/vasp/input/potcar/TYPE',
 '/vasp/input/potcar/VERSION',
 '/vasp/input

# Write VASP job

In [24]:
node_dict = {node: get_node_from_job_dict(job_dict=job_dict, node=node[1:]) for node in node_lst}

In [25]:
os.remove(file_name)

In [26]:
write_dict_to_hdf(file_name=file_name, data_dict=node_dict)

# Reload job

In [27]:
job = pr.load(job.job_name)

In [28]:
job.input.incar

,Parameter,Value,Comment
0,SYSTEM,vasp,jobname
1,PREC,Accurate,
2,ALGO,Fast,
3,LREAL,False,
4,LWAVE,False,
5,LORBIT,0,
6,ISIF,2,
7,IBRION,2,
8,NELM,400,
9,NSW,100,


In [29]:
job.input.kpoints

,Parameter,Value,Comment
0,,Kpoints file generated with pyiron_atomistics,
1,,0,
2,,Monkhorst_Pack,
3,,4 4 4,
4,,0 0 0,


In [30]:
job.input.potcar

,Parameter,Value,Comment
0,xc,GGA,"LDA, GGA"


In [31]:
job.output.energy_tot

array([-14.7459202])

In [32]:
job.output.positions

array([[[0.   , 0.   , 0.   ],
        [0.   , 2.025, 2.025],
        [2.025, 0.   , 2.025],
        [2.025, 2.025, 0.   ]]])